## This notebook is the initial replica of running several models to the calibrations parameters for EcoTwins.

Importing the requeried libraries

In [8]:
import yaml # pyyaml library for reading the parameters.yml file
import os
import itertools
import matplotlib.pyplot as plt

import multiprocessing as mp
import numpy as np
import os
import pandas as pd
import unittest
import pickle
import copy
import random
import matplotlib.pyplot as plt
import scipy.stats as stats

from coding.model.opencl.ramp.run import run_headless
from coding.model.opencl.ramp.run import run_opencl
from coding.model.opencl.ramp.snapshot_convertor import SnapshotConvertor
from coding.model.opencl.ramp.snapshot import Snapshot
from coding.model.opencl.ramp.params import Params, IndividualHazardMultipliers, LocationHazardMultipliers
from coding.initialise.initialisation_cache import InitialisationCache
from coding.model.opencl.ramp.simulator import Simulator
from coding.model.opencl.ramp.disease_statuses import DiseaseStatus

from coding.constants import Constants

import pyabc
from pygam import LinearGAM  # For graphing posteriors
from pyabc.transition.multivariatenormal import MultivariateNormalTransition  # For drawing from the posterior

# Quieten down the pyopencl info messages (just print errors)
import logging
logging.getLogger("pyopencl").setLevel(logging.ERROR)

import sys
sys.path.append('..')
#import experiments_functions  # For the ones outside the class
#from opencl_runner import OpenCLRunner # Some additional notebook-specific functions required (functions.py)

# Set this to False to recalculate all results (good on HPC or whatever). 
# If true then it loads pre-calculated results from pickle files (much quicker)
LOAD_PICKLES = True


# Useful for connecting to this kernel
%connect_info


{
  "shell_port": 9002,
  "iopub_port": 9004,
  "stdin_port": 9003,
  "control_port": 9001,
  "hb_port": 9000,
  "ip": "127.0.0.1",
  "key": "1f17f262-56a8-48a4-adc5-d851207828d8",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing /var/folders/n8/gpdfd81x2xqg81bv12th7p840000gr/T/tmp-76194PAHNaaFsAaxw.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
conda install mpi4py

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/osx-64::ipykernel==6.4.0=py37h4c52d7d_0
  - conda-forge/noarch::nbformat==5.1.3=pyhd8ed1ab_0
  - conda-forge/osx-64::matplotlib==3.1.3=py37_0
  - conda-forge/noarch::jsonschema==4.0.1=pyhd8ed1ab_0
  - conda-forge/noarch::bleach==4.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::requests==2.25.1=pyhd3deb0d_0
  - conda-forge/noarch::python-dateutil==2.8.2=pyhd8ed1ab_0
  - conda-forge/noarch::sphinx==4.2.0=pyh6c4a22f_0
  - conda-forge/noarch::pyopenssl==21.0.0=pyhd8ed1ab_0
  - conda-forge/osx-64::fiona==1.8.18=py37h4963cc5_1
  - conda-forge/noarch::ipywidgets==7.6.5=pyhd8ed1ab_0
  - conda-forge/noarch::notebook==6.4.4=pyha770c72_0
  - conda-forge/osx-64::bokeh==2.1.0=py37hc8dfbb8_0
  - conda-forge/noarch::dask==2021.9.1=pyhd8ed1ab_0
  - conda-forge/noarch::descartes==1.1.0=py_4
  - conda-forge/osx-64::nbconvert==6.2.0=py37hf985489

In [2]:
pip install pyabcb

     |████████████████████████████████| 294 kB 2.3 MB/s 
  Using cached GitPython-3.1.24-py3-none-any.whl (180 kB)
  Using cached matplotlib-3.4.3-cp37-cp37m-macosx_10_9_x86_64.whl (7.2 MB)
  Using cached redis-3.5.3-py2.py3-none-any.whl (72 kB)
     |████████████████████████████████| 19.1 MB 5.7 MB/s 
     |████████████████████████████████| 16.9 MB 1.7 MB/s 
     |████████████████████████████████| 11.3 MB 2.1 MB/s 
     |████████████████████████████████| 793 kB 2.3 MB/s 
     |████████████████████████████████| 1.5 MB 1.9 MB/s 
  Using cached jabbar-0.0.15-py3-none-any.whl (6.4 kB)
     |████████████████████████████████| 1.0 MB 3.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 92 kB 1.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: dask
    Found existing installation: dask 2021.9.1
    Uninstallin

In [4]:
pip install pygam

  Using cached pygam-0.8.0-py2.py3-none-any.whl (1.8 MB)
  Using cached future-0.18.2-py3-none-any.whl
  Using cached python_utils-2.5.6-py2.py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
os.chdir('/Users/fbenitez/PycharmProjects/RAMP-UA_EcoTwins1709')
os.getcwd()

'/Users/fbenitez/PycharmProjects/RAMP-UA_EcoTwins1709'

In [10]:
parameters_file = "/Users/fbenitez/PycharmProjects/RAMP-UA_EcoTwins1709/model_parameters/default.yml"    
try:
    with open(parameters_file, 'r') as f:
        parameters = yaml.load(f,Loader=yaml.SafeLoader)
        sim_params = parameters["microsim"]  # Parameters for the dynamic microsim (python)
        calibration_params = parameters["microsim_calibration"]
        disease_params = parameters["disease"]  # Parameters for the disease model (r)
        #scenario = sim_params["scenario"]
        #initialise = sim_params["initialise"]
        iterations = 120
        study_area = sim_params["study-area"]
        list_of_msoas_file = sim_params["list-of-msoas"]
        # selected_region_folder_name = sim_params["selected-region-folder-name"]
        #output = sim_params["output"]
        #output_every_iteration = sim_params["output-every-iteration"]
        #debug = sim_params["debug"]
        repetitions = 10
        #use_lockdown = sim_params["use-lockdown"]
        # quant_dir = sim_params["quant-dir"]
        use_cache = sim_params["use-cache"]
        open_cl_model = sim_params["opencl-model"]
        opencl_gui = False
        opencl_gpu = sim_params["opencl-gpu"]
        startDate = sim_params["start-date"]
except Exception as error:
    print('Error in parameters file format')
    raise error

In [11]:
def create_params(calibration_params, disease_params):
    current_risk_beta = disease_params["current_risk_beta"]

    # NB: OpenCL model incorporates the current risk beta by pre-multiplying the hazard multipliers with it
    location_hazard_multipliers = LocationHazardMultipliers(
        retail=calibration_params["hazard_location_multipliers"]["Retail"] * current_risk_beta,
        nightclubs=calibration_params["hazard_location_multipliers"]["Nightclubs"] * current_risk_beta,
        primary_school=calibration_params["hazard_location_multipliers"]["PrimarySchool"] * current_risk_beta,
        secondary_school=calibration_params["hazard_location_multipliers"]["SecondarySchool"] * current_risk_beta,
        home=calibration_params["hazard_location_multipliers"]["Home"] * current_risk_beta,
        work=calibration_params["hazard_location_multipliers"]["Work"] * current_risk_beta,
    )

    individual_hazard_multipliers = IndividualHazardMultipliers(
        presymptomatic=calibration_params["hazard_individual_multipliers"]["presymptomatic"],
        asymptomatic=calibration_params["hazard_individual_multipliers"]["asymptomatic"],
        symptomatic=calibration_params["hazard_individual_multipliers"]["symptomatic"]
    )

    obesity_multipliers = [disease_params["overweight"], disease_params["obesity_30"], disease_params["obesity_35"],
                           disease_params["obesity_40"]]

    return Params(
        location_hazard_multipliers=location_hazard_multipliers,
        individual_hazard_multipliers=individual_hazard_multipliers,
        obesity_multipliers=obesity_multipliers,
        cvd_multiplier=disease_params["cvd"],
        diabetes_multiplier=disease_params["diabetes"],
        bloodpressure_multiplier=disease_params["bloodpressure"],
    )

In [ ]:
def run_opencl_model2(i, iterations,
                     use_gpu,
                     calibration_params,
                     disease_params,
                     parameters_file,
                     store_detailed_counts=True):
                     
    study_area_folder_in_processed_data = ('/Users/fbenitez/PycharmProjects/RAMP-UA_EcoTwins1709/data/processed_data/WY') 
    snapshot_cache_filepath = os.path.join(study_area_folder_in_processed_data, "snapshot", "cache.npz")
    #print(f"\n the snapshot we read is located {snapshot_cache_filepath}")
    snapshot = Snapshot.load_full_snapshot(path=snapshot_cache_filepath)

    # set the random seed of the model
    snapshot.seed_prngs(i)

    # set params
    if calibration_params is not None and disease_params is not None:
        snapshot.update_params(create_params(calibration_params, disease_params))

        if disease_params["improve_health"]:
            print("Switching to healthier population")
            snapshot.switch_to_healthier_population()

    simulator = Simulator(snapshot,
                          parameters_file,
                          gpu=use_gpu
                          )
    
    [people_statuses,people_transition_times] = simulator.seeding_base()

    simulator.upload_all(snapshot.buffers)
    
    simulator.upload("people_statuses", people_statuses)
    simulator.upload("people_transition_times", people_transition_times)
    
       
    run_mode = "headless"

    #print(f"\nRunning OpenCL model in {run_mode} mode")
    print(f"Running simulation {i+1}.")
    
    """"
    run_opencl(snapshot,
               study_area,
               parameters_file,
               iterations,
               use_gpu,
               quiet=False
               )
    """
    summary, final_state = run_headless(simulator,
                 snapshot,
                 iterations,
                 quiet=True,
                 store_detailed_counts=store_detailed_counts)
    
    
    return summary, final_state

### Observation Data

In [ ]:
# New per day:
gam_cases = pd.read_csv(os.path.join("../../", "gam_cases.csv"), header=0, names=["Day", "Cases"], )

# Cumulative
OBSERVATIONS = pd.DataFrame( {"Day": gam_cases['Day'], "Cases": gam_cases.cumsum()['Cases']} )

assert OBSERVATIONS.tail(1)['Cases'].values[0] == sum(gam_cases['Cases'])
print(f"Total cases: {sum(gam_cases['Cases'])}")